In [1]:
import os
import shutil
import numpy as np
import json
import pandas as pd
import plotly.express as px
import datetime

from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize

import tensorflow as tf
from tensorflow.keras.applications import VGG16, InceptionV3, ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization, Add, ReLU, Lambda
from tensorflow.keras.models import Model,load_model

AUTOTUNE = tf.data.AUTOTUNE
IMG_SIZE = (128, 128)
EMBED_DIM = 128
BATCH_SIZE = 64
EPOCHS = 150
TEMPERATURE = 0.05


2025-11-29 11:32:18.736410: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764415938.978035      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764415939.043266      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [2]:
def contar_archivos_en_carpetas(directorio):
    # Recorre todas las carpetas dentro del directorio
    for carpeta in os.listdir(directorio):
        ruta_carpeta = os.path.join(directorio, carpeta)
        if os.path.isdir(ruta_carpeta):
            # Cuenta solo archivos (no subcarpetas)
            archivos = [f for f in os.listdir(ruta_carpeta) 
                        if os.path.isfile(os.path.join(ruta_carpeta, f))]
            print(f"Carpeta: {carpeta} -> {len(archivos)} archivos")

# Ejemplo de uso
directorio_base = "/kaggle/input/hampreprocessed/malignas_classes/train"  # Cambia esto por tu ruta
contar_archivos_en_carpetas(directorio_base)


Carpeta: mel -> 948 archivos
Carpeta: akiec -> 281 archivos
Carpeta: bcc -> 442 archivos


In [3]:
def contar_archivos_por_clase(directorio_base):
    clases_totales = {}  # acumulador por clase

    for conjunto in ["train", "test"]:
        ruta_conjunto = os.path.join(directorio_base, conjunto)
        if not os.path.exists(ruta_conjunto):
            print(f"No existe la carpeta: {ruta_conjunto}")
            continue

        print(f"\nConjunto: {conjunto}")
        for carpeta in os.listdir(ruta_conjunto):
            ruta_carpeta = os.path.join(ruta_conjunto, carpeta)
            if os.path.isdir(ruta_carpeta):
                archivos = [f for f in os.listdir(ruta_carpeta) 
                            if os.path.isfile(os.path.join(ruta_carpeta, f))]
                cantidad = len(archivos)
                print(f"  Carpeta: {carpeta} -> {cantidad} archivos")

                # acumular por clase
                if carpeta not in clases_totales:
                    clases_totales[carpeta] = 0
                clases_totales[carpeta] += cantidad

    # Mostrar suma total por clase
    print("\nSuma total por clase (train + test):")
    for clase, total in clases_totales.items():
        print(f"  {clase} -> {total} archivos")

# Ejemplo de uso
directorio_base = "/kaggle/input/hampreprocessed/malignas_classes"  # Ruta base que contiene train y test
contar_archivos_por_clase(directorio_base)



Conjunto: train
  Carpeta: benignas -> 6855 archivos
  Carpeta: malignas -> 1662 archivos

Conjunto: test
  Carpeta: benignas -> 807 archivos
  Carpeta: malignas -> 195 archivos

Suma total por clase (train + test):
  benignas -> 7662 archivos
  malignas -> 1857 archivos


## Importación de datos

In [3]:
data_dir = "/kaggle/input/hampreprocessed/malignas_classes/train"
val_dir = "/kaggle/input/hampreprocessed/malignas_classes/val"

def get_generators(data_dir, val_dir, preprocess_fn, target_size=(224, 224), batch_size=256):
    datagen = ImageDataGenerator(
        preprocessing_function=preprocess_fn,
        rotation_range=60,
        width_shift_range=0.3,
        height_shift_range=0.3,
        zoom_range=0.12,
        brightness_range=[0.8, 1.2],
        shear_range=0.2,
        vertical_flip=True,
        horizontal_flip=True
    )

    train_generator = datagen.flow_from_directory(
        data_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True
    )

    val_generator = ImageDataGenerator(preprocessing_function=preprocess_fn).flow_from_directory(
        val_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False
    )

    return train_generator, val_generator

train_generator, val_generator = get_generators(data_dir, val_dir, lambda x: x)
print(pd.Series(val_generator.classes).value_counts())
print(pd.Series(train_generator.classes).value_counts())

labels = train_generator.classes  

# Calculamos los pesos
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(labels),
    y=labels
)

# Lo convertimos en diccionario para Keras
class_weights = dict(enumerate(class_weights))
print(class_weights)

Found 1671 images belonging to 3 classes.
Found 88 images belonging to 3 classes.
2    54
1    19
0    15
Name: count, dtype: int64
2    948
1    442
0    281
Name: count, dtype: int64
{0: 1.9822064056939501, 1: 1.260180995475113, 2: 0.5875527426160337}


In [4]:
train_generator, val_generator = get_generators(data_dir, val_dir, lambda x: x/255., target_size=IMG_SIZE, batch_size=BATCH_SIZE)
num_classes = len(train_generator.class_indices)
class_names = list(train_generator.class_indices.keys())

Found 1671 images belonging to 3 classes.
Found 88 images belonging to 3 classes.


## Modelo generador de embeddings

In [5]:
def create_transfer_model(base_model_fn, input_shape=(224,224,3), n_classes=1, dropout=0.2, trainable_layers=0):
    base = base_model_fn(
        include_top=False,
        weights="imagenet",
        input_shape=input_shape
    )

    # Congelamos todas las capas primero
    base.trainable = False

    # Si se especifican capas entrenables, las activamos desde el final
    if trainable_layers > 0:
        for layer in base.layers[-trainable_layers:]:
            layer.trainable = True

    x = GlobalAveragePooling2D()(base.output)
    x = Dense(128, activation="relu")(x)
    x = Dense(64, activation="relu")(x)
    x = Dropout(dropout)(x)
    output = Dense(n_classes, activation="sigmoid")(x)

    model = Model(inputs=base.input, outputs=output)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
        loss="binary_crossentropy",
        metrics=["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
    )
    return model

In [6]:
def contrastive_encoder(input_shape=(IMG_SIZE[0],IMG_SIZE[1],3), embedding_dim=EMBED_DIM):
    inputs = Input(shape=input_shape)

    # Bloque 1
    x = Conv2D(64, 3, padding='same', use_bias=False)(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv2D(64, 3, padding='same', use_bias=False)(x)
    x = BatchNormalization()(x)
    s = Conv2D(64, 1, padding='same', use_bias=False)(inputs)
    s = BatchNormalization()(s)
    x = Add()([x, s])
    x = ReLU()(x)
    x = MaxPooling2D()(x)

    # Bloque 2
    y = Conv2D(128, 3, padding='same', use_bias=False)(x)
    y = BatchNormalization()(y)
    y = ReLU()(y)
    y = Conv2D(128, 3, padding='same', use_bias=False)(y)
    y = BatchNormalization()(y)
    s2 = Conv2D(128, 1, padding='same', use_bias=False)(x)
    s2 = BatchNormalization()(s2)
    y = Add()([y, s2])
    y = ReLU()(y)
    y = MaxPooling2D()(y)

    # Bloque 3
    z = Conv2D(256, 3, padding='same', use_bias=False)(y)
    z = BatchNormalization()(z)
    z = ReLU()(z)
    z = Conv2D(256, 3, padding='same', use_bias=False)(z)
    z = BatchNormalization()(z)
    s3 = Conv2D(256, 1, padding='same', use_bias=False)(y)
    s3 = BatchNormalization()(s3)
    z = Add()([z, s3])
    z = ReLU()(z)

    z = GlobalAveragePooling2D()(z)
    z = Dense(512, activation='relu')(z)
    z = BatchNormalization()(z)

    # Proyección (cabeza contrastiva)timestamps
    p = Dense(embedding_dim, activation='relu')(z)
    p = Dense(embedding_dim)(p)
    outputs = Lambda(
    lambda t: tf.math.l2_normalize(t, axis=1),
    name="proj_norm",
    output_shape=(embedding_dim,))(p)


    return Model(inputs, outputs, name="ContrastiveEncoder")


In [7]:
class SupConLoss(tf.keras.losses.Loss):
    def __init__(self, temperature=0.1, name="supcon"):
        super().__init__(name=name)
        self.temperature = temperature

    def call(self, y_true, features):
        """
        SupConLoss implementation.
        Args:
            y_true: [batch] integer class labels (not one-hot).
            features: [batch, dim] embeddings.
        """
        # Normalize embeddings
        features = tf.math.l2_normalize(features, axis=1)
        batch_size = tf.shape(features)[0]

        # Similarity matrix
        sim = tf.matmul(features, features, transpose_b=True)  # [B, B]
        sim = sim / self.temperature

        # Ensure labels are integers, not one-hot
        if y_true.shape.ndims > 1 and y_true.shape[-1] > 1:
            y_true = tf.argmax(y_true, axis=-1)

        labels = tf.reshape(y_true, [-1, 1])  # [B, 1]
        mask = tf.equal(labels, tf.transpose(labels))  # [B, B]
        mask = tf.cast(mask, tf.float32)

        # Remove self-contrast
        eye = tf.eye(batch_size, dtype=tf.float32)
        logits_mask = tf.ones_like(mask) - eye
        mask = mask * logits_mask

        # Log-softmax denominator excluding self
        sim_max = tf.reduce_max(sim, axis=1, keepdims=True)
        sim = sim - sim_max
        exp_sim = tf.exp(sim) * logits_mask
        denom = tf.reduce_sum(exp_sim, axis=1, keepdims=True) + 1e-9
        log_prob = sim - tf.math.log(denom)

        # Average log-prob of positives per anchor
        pos_count = tf.reduce_sum(mask, axis=1) + 1e-9
        mean_log_pos = tf.reduce_sum(mask * log_prob, axis=1) / pos_count

        loss = -tf.reduce_mean(mean_log_pos)
        return loss


## Entrenar representaciones

In [8]:
def evaluate_embeddings(model, train_generator, val_generator, k=3, train_steps=50, val_steps=50):
    """Entrena KNN y NearestCentroid con embeddings de train y evalúa en val."""
    # --- Embeddings de train ---
    train_embeds, train_labels = [], []
    for _ in range(train_steps):
        images, labels = next(train_generator)
        embeds = model(images, training=False).numpy()
        train_embeds.append(embeds)
        train_labels.append(np.argmax(labels, axis=1))  # convertir one-hot a entero
    X_train = np.concatenate(train_embeds, axis=0)
    y_train = np.concatenate(train_labels, axis=0)

    # --- Embeddings de val ---
    val_embeds, val_labels = [], []
    for _ in range(val_steps):
        images, labels = next(val_generator)
        embeds = model(images, training=False).numpy()
        val_embeds.append(embeds)
        val_labels.append(np.argmax(labels, axis=1))
    X_val = np.concatenate(val_embeds, axis=0)
    y_val = np.concatenate(val_labels, axis=0)

    # --- KNN ---
    knn = KNeighborsClassifier(n_neighbors=k, metric="cosine", weights="distance")
    knn.fit(X_train, y_train)
    y_pred_knn = knn.predict(X_val)
    acc_knn = accuracy_score(y_val, y_pred_knn)

    # --- Nearest Centroid ---
    centroid = NearestCentroid(metric="cosine")
    centroid.fit(X_train, y_train)
    y_pred_centroid = centroid.predict(X_val)
    acc_centroid = accuracy_score(y_val, y_pred_centroid)

    print(f"k-NN Acc: {acc_knn:.4f}")
    print(f"Nearest Centroid Acc: {acc_centroid:.4f}")

    return acc_knn, acc_centroid


def train_supcon(model, train_generator, val_generator, loss_fn, optimizer, epochs=50, accumulate_steps=2):
    steps_per_epoch = train_generator.samples // train_generator.batch_size
    validation_steps = val_generator.samples // val_generator.batch_size

    train_loss = tf.keras.metrics.Mean(name="train_loss")
    val_loss = tf.keras.metrics.Mean(name="val_loss")

    for epoch in range(epochs):
        train_loss.reset_state()
        val_loss.reset_state()

        # Training
        accum_grads = [tf.zeros_like(var) for var in model.trainable_variables]
        step_count = 0

        for _ in range(steps_per_epoch):
            images, labels = next(train_generator)

            with tf.GradientTape() as tape:
                embeddings = model(images, training=True)
                loss = loss_fn(labels, embeddings)

            grads = tape.gradient(loss, model.trainable_variables)
            accum_grads = [accum + grad for accum, grad in zip(accum_grads, grads)]
            step_count += 1
            train_loss.update_state(loss)

            # Apply gradients every `accumulate_steps`
            if step_count % accumulate_steps == 0:
                mean_grads = [accum / accumulate_steps for accum in accum_grads]
                optimizer.apply_gradients(zip(mean_grads, model.trainable_variables))
                accum_grads = [tf.zeros_like(var) for var in model.trainable_variables]

        # Validation
        for _ in range(validation_steps):
            images, labels = next(val_generator)
            embeddings = model(images, training=False)
            loss = loss_fn(labels, embeddings)
            val_loss.update_state(loss)

        print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss.result():.4f} - Val Loss: {val_loss.result():.4f}")

        # Evaluation every 5 epochs
        if (epoch + 1) % 5 == 0:
            acc_knn, acc_centroid = evaluate_embeddings(
                model, train_generator, val_generator,
                k=3, train_steps=steps_per_epoch, val_steps=validation_steps
            )
            print(f"k-NN Acc: {acc_knn:.4f}")
            print(f"Nearest Centroid Acc: {acc_centroid:.4f}")


In [11]:
encoder = contrastive_encoder(embedding_dim=EMBED_DIM)
encoder.trainable = True
loss_fn = SupConLoss(temperature=TEMPERATURE)
optimizer = Adam(learning_rate=8e-4)
train_supcon(encoder, train_generator, val_generator, loss_fn, optimizer, epochs=5)
# Current timestamp
timestamp = datetime.datetime.now().strftime("%m_%d_h%H_%M")

encoder.save(f"encoder_{timestamp}.keras")

Epoch 1/5 - Train Loss: 3.7674 - Val Loss: 4.2446
Epoch 2/5 - Train Loss: 3.7599 - Val Loss: 3.3043
Epoch 3/5 - Train Loss: 3.7808 - Val Loss: 3.8724
Epoch 4/5 - Train Loss: 3.7725 - Val Loss: 3.3122
Epoch 5/5 - Train Loss: 3.7239 - Val Loss: 4.0184
k-NN Acc: 1.0000
Nearest Centroid Acc: 1.0000
k-NN Acc: 1.0000
Nearest Centroid Acc: 1.0000


/usr/local/lib/python3.11/dist-packages/sklearn/neighbors/_nearest_centroid.py:179: UserWarning: Averaging for metrics other than euclidean and manhattan not supported. The average is set to be the mean.
  warnings.warn(


In [15]:
from keras.applications.vgg16 import preprocess_input as vgg16_preprocess
train_generator, val_generator = get_generators(data_dir,val_dir, vgg16_preprocess)
vgg_encoder = create_transfer_model(VGG16, trainable_layers= 4)
encoder.trainable = True
loss_fn = SupConLoss(temperature=TEMPERATURE)
optimizer = Adam(learning_rate=8e-4)
train_supcon(vgg_encoder, train_generator, val_generator, loss_fn, optimizer, epochs=25)

# Guardar
timestamp = datetime.datetime.now().strftime("%m_%d_%H:%M")
vgg_encoder.save(os.path.join(save_dir, f"vgg16_encoder_{timestamp}.keras"))

Found 1671 images belonging to 3 classes.
Found 88 images belonging to 3 classes.
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


ResourceExhaustedError: Exception encountered when calling Conv2D.call().

[1m{{function_node __wrapped__Conv2D_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[256,64,224,224] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Conv2D][0m

Arguments received by Conv2D.call():
  • inputs=tf.Tensor(shape=(256, 224, 224, 64), dtype=float32)

## Calcular centroides de clases

In [15]:
def compute_centroids(encoder, generator):
    """
    Calcula centroides de clase a partir de un generator de Keras.
    Devuelve un dict {class_index: centroid_vector}.
    """
    embeds, labels = [], []
    for i in range(len(generator)):
        x_batch, y_batch = generator[i]
        e = encoder.predict(x_batch, verbose=0)
        e = normalize(e)  # normalizar embeddings fila a fila
        embeds.append(e)
        labels.append(np.argmax(y_batch, axis=1))  # convertir one-hot a entero
    # print(pd.Series(labels).value_counts())

    embeds = np.concatenate(embeds)
    labels = np.concatenate(labels)

    centroids = {}
    for c in np.unique(labels):
        class_embeds = embeds[labels == c]
        centroid = class_embeds.mean(axis=0)
        centroid = centroid / np.linalg.norm(centroid)  # normalizar centroide
        centroids[int(c)] = centroid.tolist()  # convertir a lista para JSON

    return centroids

def save_centroids(centroids, filename=None):
    if not filename:
        timestamp = datetime.datetime.now().strftime("%m_%d_h%H_%M")
        filename = f"centroids_{timestamp}.json"
    with open(filename, "w") as f:
        json.dump(centroids, f)

def load_centroids(filename="centroids.json"):
    with open(filename, "r") as f:
        centroids = json.load(f)
    # convertir a numpy arrays
    centroids = {int(k): np.array(v) for k, v in centroids.items()}
    return centroids

In [16]:
centroids = compute_centroids(encoder, train_generator)
save_centroids(centroids)

I0000 00:00:1764167245.090675     770 service.cc:148] XLA service 0x7b5600005d80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1764167245.091266     770 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1764167248.198486     770 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


## Evaluar

In [17]:
def predict_class(encoder, x, centroids, probs=False):
    """
    Predice la clase de una sola imagen x usando centroides.
    """
    e = encoder.predict(np.expand_dims(x, axis=0), verbose=0)
    e = normalize(e)  # normalizar embedding
    sims = {c: np.dot(e, centroids[c]) for c in centroids}
    if probs:
        return sims
    return max(sims, key=sims.get)  # clase con mayor similitud

def evaluate_accuracy(encoder, val_generator, centroids):
    """
    Calcula el accuracy del val_generator usando centroides.
    """
    y_true, y_pred = [], []

    for i in range(len(val_generator)):
        x_batch, y_batch = val_generator[i]
        labels = np.argmax(y_batch, axis=1)  # convertir one-hot a enteros

        for j in range(len(x_batch)):
            pred = predict_class(encoder, x_batch[j], centroids)
            y_true.append(labels[j])
            y_pred.append(pred)

    acc = accuracy_score(y_true, y_pred)
    print(f"Accuracy en val_generator: {acc:.4f}")
    return acc

In [18]:
evaluate_accuracy(encoder, val_generator, centroids)

Accuracy en val_generator: 0.6373


0.6373429084380611

## KNN as classifier

In [10]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np

def train_knn(encoder, train_generator, k=10):
    """
    Entrena un KNN sobre los embeddings del train_generator.
    Devuelve el clasificador entrenado.
    """
    X, y = [], []
    for i in range(len(train_generator)):
        x_batch, y_batch = train_generator[i]
        e = encoder.predict(x_batch, verbose=0)
        e = e / np.linalg.norm(e, axis=1, keepdims=True)  # normalizar embeddings
        X.append(e)
        y.append(np.argmax(y_batch, axis=1))

    X = np.concatenate(X)
    y = np.concatenate(y)

    knn = KNeighborsClassifier(n_neighbors=k, metric="cosine")
    knn.fit(X, y)
    return knn

def evaluate_knn(encoder, val_generator, knn):
    """
    Evalúa un KNN entrenado sobre el val_generator.
    """
    X_val, y_val = [], []
    for i in range(len(val_generator)):
        x_batch, y_batch = val_generator[i]
        e = encoder.predict(x_batch, verbose=0)
        e = e / np.linalg.norm(e, axis=1, keepdims=True)
        X_val.append(e)
        y_val.append(np.argmax(y_batch, axis=1))

    X_val = np.concatenate(X_val)
    y_val = np.concatenate(y_val)

    y_pred = knn.predict(X_val)
    acc = accuracy_score(y_val, y_pred)
    print(f"Accuracy en val_generator con KNN: {acc:.4f}")
    return acc

In [12]:
encoder = contrastive_encoder(
    input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3),
    embedding_dim=EMBED_DIM
)

# Cargar solo los pesos entrenados
encoder.load_weights("/kaggle/working/encoder_finetuned_11_26_h15_34.keras")

knn = train_knn(encoder, train_generator)
import pickle

# Guardar el modelo entrenado en un archivo
timestamp = datetime.datetime.now().strftime("%m_%d_h%H_%M")
with open(f"knn_model_{timestamp}.pkl", "wb") as f:
    pickle.dump(knn, f)
print("Entrenado")
evaluate_knn(encoder, val_generator, knn)

Entrenado
Accuracy en val_generator con KNN: 0.7841


0.7840909090909091

In [14]:
encoder = contrastive_encoder(
    input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3),
    embedding_dim=EMBED_DIM
)

# Cargar solo los pesos entrenados
encoder.load_weights("/kaggle/working/encoder_11_26_h14_27.keras")
knn = train_knn(encoder, train_generator, k=3)
import pickle

# Guardar el modelo entrenado en un archivo
timestamp = datetime.datetime.now().strftime("%m_%d_h%H_%M")
with open(f"knn_model_{timestamp}.pkl", "wb") as f:
    pickle.dump(knn, f)
print("Entrenado")
evaluate_knn(encoder, val_generator, knn)

Entrenado
Accuracy en val_generator con KNN: 0.8295


0.8295454545454546

In [10]:
encoder.trainable = False
x = encoder.output
clf = Dense(num_classes, activation="softmax")(x)
classifier = Model(encoder.input, clf)
classifier.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [13]:
labels = train_generator.classes
print(pd.Series(labels).value_counts())
print(num_classes)
class_weights = dict(enumerate(compute_class_weight(
    class_weight="balanced",
    classes=np.unique(labels),
    y=labels
)))

classifier.compile(optimizer=Adam(learning_rate=1e-5), loss="categorical_crossentropy", metrics=["accuracy"])
encoder.trainable = True
classifier.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    class_weight=class_weights
)


2    948
1    442
0    281
Name: count, dtype: int64
3
Epoch 1/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 29s 756ms/step - accuracy: 0.8555 - loss: 0.3975 - val_accuracy: 0.8523 - val_loss: 0.3850
Epoch 2/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 17s 638ms/step - accuracy: 0.8616 - loss: 0.4258 - val_accuracy: 0.8523 - val_loss: 0.3888
Epoch 3/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 17s 637ms/step - accuracy: 0.8578 - loss: 0.4094 - val_accuracy: 0.8523 - val_loss: 0.3921
Epoch 4/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 17s 653ms/step - accuracy: 0.8530 - loss: 0.3950 - val_accuracy: 0.8295 - val_loss: 0.4187
Epoch 5/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 18s 651ms/step - accuracy: 0.8671 - loss: 0.4082 - val_accuracy: 0.8409 - val_loss: 0.3976
Epoch 6/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 18s 656ms/step - accuracy: 0.8726 - loss: 0.3902 - val_accuracy: 0.8409 - val_loss: 0.3950
Epoch 7/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 17s 628ms/step - accuracy: 0.8829 - loss: 0.3808 - val_accuracy: 0.8182 - val_loss: 0.4450
Epoch 8/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 17s 

In [14]:
# Current timestamp
timestamp = datetime.datetime.now().strftime("%m_%d_h%H_%M")

encoder.save(f"encoder_finetuned_{timestamp}.keras")

In [15]:
# Current timestamp
timestamp = datetime.datetime.now().strftime("%m_%d_h%H_%M")

classifier.save(f"classifier_{timestamp}.keras")

In [18]:
def visualize_embeddings_3d(model, val_generator, class_names, method="tsne"):
    # 1. Calcular cuántos pasos tiene la validación
    validation_steps = val_generator.samples // val_generator.batch_size

    embs, labs = [], []
    for _ in range(validation_steps):
        images, labels = next(val_generator)
        e = model(images, training=False).numpy()
        embs.append(e)
        labs.append(labels)

    X = np.concatenate(embs, axis=0)
    y = np.concatenate(labs, axis=0)

    # 2. Reducir a 3D
    if method == "tsne":
        reducer = TSNE(n_components=3, perplexity=30, learning_rate=200, random_state=42)
    else:
        reducer = PCA(n_components=3)
    X_reduced = reducer.fit_transform(X)

    # 3. Visualizar con Plotly
    fig = px.scatter_3d(
        x=X_reduced[:,0], y=X_reduced[:,1], z=X_reduced[:,2],
        color=[class_names[i] for i in y],
        title=f"Embeddings en 3D ({method.upper()})",
        opacity=0.7
    )
    fig.show()

In [ ]:
visualize_embeddings_3d(encoder, val_generator, class_names)

In [19]:
def euclidean_distance(vects):
    """Find the Euclidean distance between two vectors.

    Arguments:
        vects: List containing two tensors of same length.

    Returns:
        Tensor containing euclidean distance
        (as floating point value) between vectors.
    """

    x, y = vects
    sum_square = tf.math.reduce_sum(tf.math.square(x - y), axis=1, keepdims=True)
    return tf.math.sqrt(tf.math.maximum(sum_square, tf.keras.backend.epsilon()))


def build_siamese_network(encoder, input_shape):
    input_a = Input(shape=input_shape)
    input_b = Input(shape=input_shape)

    encoded_a = encoder(input_a)
    encoded_b = encoder(input_b)

    # Distancia euclídea entre embeddings
    distance = Lambda(euclidean_distance)([encoded_a, encoded_b])

    # Una neurona con sigmoide decide si son similares
    outputs = Dense(1, activation="sigmoid")(distance)

    siamese_net = Model([input_a, input_b], outputs)
    return siamese_net

In [24]:
encoder = contrastive_encoder(
    input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3),
    embedding_dim=EMBED_DIM
)

# Cargar solo los pesos entrenados
encoder.load_weights("/kaggle/working/encoder_11_26_h14_27.keras")

encoder.trainable=True
siamese_model = build_siamese_network(encoder, (IMG_SIZE[0], IMG_SIZE[0], 3))
siamese_model.compile(loss="binary_crossentropy", optimizer=Adam(1e-5), metrics=["accuracy"])
siamese_model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_5       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ContrastiveEncoder  │ (None, 128)       │  1,406,848 │ input_layer_4[0]… │
│ (Functional)        │                   │            │ input_layer_5[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_1 (Lambda)   │ (None, 1)         │          0 │ ContrastiveEncod… │
│                     │                   │            │ ContrastiveEncod… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 1)         │          2 │ lambda_1[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,406,850 (5.37 MB)

 Trainable params: 1,403,138 (5.35 MB)

 Non-trainable params: 3,712 (14.50 KB)

In [22]:
import random
def make_pairs_from_generator(generator):
    """
    Crea pares de imágenes (positivos y negativos) a partir de un generator de Keras.
    
    Arguments:
        generator: un ImageDataGenerator.flow_from_directory u otro generator que devuelva (x_batch, y_batch).
    
    Returns:
        pairs: numpy array de shape (2*N, 2, H, W, C)
        labels: numpy array binario de shape (2*N,)
    """
    # --- 1. Extraer todas las imágenes y etiquetas del generator ---
    all_images, all_labels = [], []
    for i in range(len(generator)):
        x_batch, y_batch = generator[i]
        all_images.append(x_batch)
        all_labels.append(np.argmax(y_batch, axis=1))  # convertir one-hot a entero
    
    x = np.concatenate(all_images, axis=0)
    y = np.concatenate(all_labels, axis=0)

    # --- 2. Crear índices por clase ---
    num_classes = np.max(y) + 1
    digit_indices = [np.where(y == i)[0] for i in range(num_classes)]

    pairs = []
    labels = []

    # --- 3. Generar pares ---
    for idx1 in range(len(x)):
        x1 = x[idx1]
        label1 = y[idx1]

        # Par positivo (misma clase)
        idx2 = random.choice(digit_indices[label1])
        x2 = x[idx2]
        pairs.append([x1, x2])
        labels.append(1)  # aquí 1 = misma clase

        # Par negativo (clase distinta)
        label2 = random.randint(0, num_classes - 1)
        while label2 == label1:
            label2 = random.randint(0, num_classes - 1)
        idx2 = random.choice(digit_indices[label2])
        x2 = x[idx2]
        pairs.append([x1, x2])
        labels.append(0)  # aquí 0 = distinta clase

    return np.array(pairs), np.array(labels).astype("float32")

pairs_train, labels_train = make_pairs_from_generator(train_generator)
pairs_val, labels_val = make_pairs_from_generator(val_generator)

In [25]:
history = siamese_model.fit(
    [pairs_train[:,0], pairs_train[:,1]], labels_train,
    validation_data=([pairs_val[:,0], pairs_val[:,1]], labels_val),
    batch_size=BATCH_SIZE,
    epochs=100
)

Epoch 1/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 29s 319ms/step - accuracy: 0.5037 - loss: 0.6679 - val_accuracy: 0.5227 - val_loss: 0.6628
Epoch 2/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 11s 205ms/step - accuracy: 0.4992 - loss: 0.6591 - val_accuracy: 0.5227 - val_loss: 0.6604
Epoch 3/100
35/53 ━━━━━━━━━━━━━━━━━━━━ 3s 204ms/step - accuracy: 0.4868 - loss: 0.6601

KeyboardInterrupt: 